In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
from video.data import read_dataset

In [3]:
tf.enable_eager_execution()

# カテゴリ一覧

In [4]:
!gsutil cp gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv .

Copying gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [5]:
import csv

label_map = {}
with open('./vocabulary.csv', 'r') as c:
    reader = csv.reader(c)
    next(reader, None)
    for r in reader:
        label_map[r[0]] = r[3]

# 評価データセットの読み込み

In [6]:
data = read_dataset(
    'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord',
    tf.estimator.ModeKeys.EVAL,
    batch_size=5
).shuffle(1024)
data_iter = iter(data)

# 推論

In [7]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

PROJECT = 'qwiklabs-gcp-ml-83914069970b'
MODEL_NAME = 'youtube8mhal'
VERSION = 'v0'

credentials = GoogleCredentials.get_application_default()
api = discovery.build(
    "ml",
    "v1",
    credentials = credentials,
    discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json"
)

def convert_data(data_iter):
    batch, labels = next(data_iter)
    batch.pop('id')
    feature = [{'mean_rgb': r.numpy().tolist(), 'mean_audio': a.numpy().tolist()} for r, a in zip(batch['mean_rgb'], batch['mean_audio'])]
    labels = [[label_map[str(i)] for i, v in enumerate(label) if v] for label in labels]
    return feature, labels

def show_predict(prediction, label):
    print('labels:', label)
    for i, index in enumerate(prediction["predicted_topk"]):
        prob = prediction['probabilities'][index]
        print("{:>20} {}th({}:{}) probality is {}".format(
            '=' * int(prob * 20),
            (i+1),
            index,
            label_map["{}".format(index)],
            prob,
        ))
    print('')

def predict(feature, labels=[], model_name=MODEL_NAME, version=VERSION):
    request_data = {"instances": feature}
    parent = "projects/{}/models/{}/versions/{}".format(PROJECT,MODEL_NAME, VERSION)
    response = api.projects().predict(body = request_data, name = parent).execute()
    if 'predictions' not in response:
        print('ERROR:', response)
    labels += [''] * 10
    for p, l in zip(response['predictions'], labels):
        show_predict(p, l)
    return None

# 評価データからの推論

In [8]:
feature, labels = convert_data(data_iter)
predict(feature, labels=labels, model_name='youtube8mkojo', version='v1')

labels: ['Game', 'Video game', 'Cartoon', 'Fighting game', 'Street Fighter', 'Ryu (Street Fighter)', 'Street Fighter II: The World Warrior']
 =================== 1th(1864:Street Fighter II: The World Warrior) probality is 0.9997466802597046
 =================== 2th(69:Fighting game) probality is 0.9994351267814636
 =================== 3th(236:Street Fighter) probality is 0.9983654618263245
 =================== 4th(1:Video game) probality is 0.9964624047279358
 =================== 5th(0:Game) probality is 0.9929423332214355

labels: ['Dance']
    ================ 1th(13:Musical ensemble) probality is 0.823073148727417
            ======== 2th(8:Dance) probality is 0.4208844304084778
               ===== 3th(4:Musician) probality is 0.2575816512107849
                   = 4th(3:Concert) probality is 0.08166596293449402
                   = 5th(175:Parade) probality is 0.057628899812698364

labels: ['Concert']
 =================== 1th(3:Concert) probality is 0.9989173412322998
           

# GCS からの予測

In [9]:
import requests

def tag_image(file):
    job_id = 'haljob'

    file_prefix, ext = file.rsplit('.', 1)
    response = requests.get("http://35.223.176.253:8080/predict/{}/{}/{}".format(job_id, file_prefix, ext))
    feature = response.json()
    print(file)
    predict([feature], model_name='youtube8mkojo', version='v1')

In [10]:
image_list = [
    'aibo.mov',
    'art_robot.mov',
    'cycling_bridge.mp4',
    'indoor_climbing.mov',
    'mountain_climbing.mp4',
    'outdoor_climbing.mov',
    'parrot.mov',
    'qoobo.mov',
    'rabbit.mov',
    'restaurant.mov',
    'robot_arm.mov',
    'snowboarding.mov',
    'sunset.mp4',
]
for file in image_list:
    try:
        tag_image(file)
    except Exception as e:
        print('ERROR:', e)

aibo.mov
labels: 
        ============ 1th(26:Toy) probality is 0.6493330001831055
              ====== 2th(15:Animal) probality is 0.3113280236721039
               ===== 3th(738:Bowser (character)) probality is 0.2834187150001526
                ==== 4th(326:Figurine) probality is 0.2309437394142151
                ==== 5th(782:Stuffed toy) probality is 0.2085438370704651

art_robot.mov
labels: 
                   = 1th(2:Vehicle) probality is 0.06031396985054016
                     2th(26:Toy) probality is 0.03662163019180298
                     3th(1:Video game) probality is 0.03249591588973999
                     4th(0:Game) probality is 0.01539844274520874
                     5th(1750:Predator (alien)) probality is 0.009896695613861084

cycling_bridge.mp4
labels: 
 =================== 1th(871:Bridge) probality is 0.9993628263473511
                     2th(179:Weather) probality is 0.01919502019882202
                     3th(45:Train) probality is 0.0113411545753479
        